In [ ]:
using LazySets

In [ ]:
points = N -> [randn(2) for i in 1:N]
v = points(30)
hull = convex_hull(v)
typeof(hull), length(v), length(hull)

In [ ]:
n = length(Y_train)
m = length(x[1,:])
clusters = Vector{Cluster}([])
for dataId in 1:size(x,1)
    push!(clusters, Cluster(dataId, x,y))
end
clusterId = collect(1:n) #On obtient un vecteur 1,2..., qui correspond pour chaque cluster à son clusterId
distances = Vector{Distance}([])
for id1 in 1:n-1
    for id2 in id1+1:n
        push!(distances, Distance(id1, id2, x))
    end
end
sort!(distances, by = v ->v.distance)
remainingClusters=n
distanceId = 1
n_epochs = 1
c1_bis = Nothing
c2_bis = Nothing
for i in  1:n_epochs
    a=i
    distance = distances[distanceId]
    cId1 =clusterId[distance.ids[1]]
    cId2 = clusterId[distance.ids[2]]
    if cId1 != cId2
        c1 = clusters[cId1]
        c2 = clusters[cId2]
        count_classes_in_clusters = StatsBase.countmap([y[c2.dataIds]; y[c1.dataIds]])
        df = DataFrame()
        df."classes" = collect(keys(count_classes_in_clusters))
        df."count_classes" = collect(values(count_classes_in_clusters))
        df = sort(df, [:count_classes], rev=true)
        remainingClusters -=1
        merge!(c1, c2) #On merge les 2 clusters
        for id in c2.dataIds 
            clusterId[id]= cId1 #On modifie le clusterId dans la serie pour le cluster_2, on lui affecte le cluster_1
        end
        # Vider le second cluster
        if distanceId<n_epochs-10
            empty!(clusters[cId2].dataIds)
        else
            print(distanceId)
        end

    end
    distanceId += 1
end
function num_clustersmeans(x,num_clusters, maxiters = 100, tol = 1e-5)
    x = collect(eachrow(x))
    N = length(x)
    n = length(x[1])
    distances = zeros(N) # used to store the distance of each
    # point to the nearest representative.
    reps = [zeros(n) for j=1:num_clusters] # used to store representatives.

    # ’assignment’ is an array of N integers between 1 and num_clusters.
    # The initial assignment is chosen randomly.
    assignment = [ rand(1:num_clusters) for i in 1:N ]

    Jprevious = Inf # used in stopping condition

    for iter = 1:maxiters

        # Cluster j representative is average of points in cluster j.
        for j = 1:num_clusters
            group = [i for i=1:N if assignment[i] == j]
            reps[j] = sum(x[group]) / length(group);
        end;

        # For each x[i], find distance to the nearest reprepresentative
        # and its group index

        for i = 1:N
            (distances[i], assignment[i]) = findmin([norm(x[i] - reps[j]) for j = 1:num_clusters])
        end;

        # Compute clustering objective.
        J = norm(distances)^2 / N

        # Show progress and terminate if J stopped decreasing.
        println("Iteration ", iter, ": Jclust = ", J, ".")

        if iter > 1 && abs(J - Jprevious) < tol * J
            return assignment, reps
        end

        Jprevious = J
    end

end

num_clusters = 20
x = X_train
N = length(x)
n = size(x)[2]
distances = zeros(N) # used to store the distance of each
# point to the nearest representative.
reps = [zeros(n) for j=1:num_clusters] # used to store representatives.

# ’assignment’ is an array of N integers between 1 and num_clusters.
# The initial assignment is chosen randomly.
assignment = [ rand(1:num_clusters) for i in 1:N ]

Jprevious = Inf # used in stopping condition
center_clusters = rand(num_clusters, n)
